# SatPy: a tool for handling scenes and overpasses

Objects you need to know about:

* Scene
* Dataset
* DatasetId

`Scene` is a high level object that can read/modify/resample/plot satellite observations.
`Dataset` is a container for the particular observations. Child of Numpy Masked array.
`DatasetId` is a tuple with observations identifiers.

# Obtain MODIS 1KM datasets

LAADS WEB provides access to the 5 minute MODIS granules over Svalbard and Greenladn. Here we get to work with these two: 
* [2017-08-26 14:35](https://lance-modis.eosdis.nasa.gov/cgi-bin/imagery/single.cgi?image=crefl1_143.A2017238143500-2017238144000.2km.jpg)
* [2017-08-26 14:40](https://lance-modis.eosdis.nasa.gov/cgi-bin/imagery/single.cgi?image=crefl1_143.A2017238144000-2017238144500.2km.jpg)

In [ ]:
# get the data
#!wget -c ftp://ladsweb.nascom.nasa.gov/allData/61/MOD03/2017/238/MOD03*.A2017238.1435.061.2017317001541.hdf
#!wget -c ftp://ladsweb.nascom.nasa.gov/allData/61/MOD021KM/2017/238/MOD021KM.A2017238.1435.061.2017317012902.hdf
#!wget -c ftp://ladsweb.nascom.nasa.gov/allData/61/MOD03/2017/238/MOD03.A2017238.1440.061.*.hdf
#!wget -c ftp://ladsweb.nascom.nasa.gov/allData/61/MOD021KM/2017/238/MOD021KM.A2017238.1440.061.*.hdf
#!wget -c ftp://ladsweb.nascom.nasa.gov/allData/61/MOD03/2017/238/MOD03.A2017238.1300.061.*.hdf
!wget -c ftp://ladsweb.nascom.nasa.gov/allData/61/MOD021KM/2017/238/MOD021KM.A2017238.1300.061.*.hdf
 


# Selecting scenes/observations from disk

The data can be selected either directly using keyword arguments `filenames` and `reader` or indirectly using `sensor`, `basedir` and `start_time`/`end_time`.


In [ ]:
from satpy import Scene
from satpy.utils import debug_on
from datetime import datetime
import glob

debug_on()

scn = Scene(platform_name="Terra", sensor="modis",
                base_dir=".",
                reader='hdfeos_l1b',
                start_time=datetime(2017, 8, 26, 14, 35),
                end_time=datetime(2017, 8, 26, 14, 45))

#scn.load(['true_color'])
#scn.show('true_color')

In [ ]:
resampled_scene = scn.resample('sval')
resampled_scene.show('true_color')
resampled_scene.save_dataset('true_color', filename='tc.tif')

In [ ]:
from datetime import datetime
from satpy.scene import Scene
from satpy.utils import debug_on
from satpy.utils import logging_off
# debug_on()
logging_off()
start_time = datetime(2017,12,2)

scn = Scene(
        sensors='sar-c',
        reader='sar_c',
        start_time=start_time,
        base_dir="../")

# Explore what is available for the S1 scene


In [ ]:
# Available datasets
print(scn.available_dataset_names())

# Available composites
print(scn.available_composite_names())


# Specify dataset explicitly via DatasetID

In [ ]:
from satpy import DatasetID

# Give example of using DatasetID
dst_id = DatasetID(name='measurement', polarization='hv')
scn.load([dst_id])
# Load and access the data

In [ ]:
scn.datasets.keys()
scn.save_dataset(dst_id, filename='hv.tif')

# Access the data

In [ ]:
# How to access the data/coordinates?


# Resample original data to a regular grid

In [ ]:
from pyresample import geometry

area_def = geometry.AreaDefinition(area_extent=(-2056956.074, 539336.813,
                                                 1097748.327,-2586425.345),
                                   proj_dict=dict(proj="stere",
                                                   ellps="WGS84",
                                                   lat_0="90",
                                                   lon_0="20",
                                                   lat_ts="75",),
                                   x_size=1000,
                                   y_size=1000,
                                   proj_id='foo',
                                   area_id='bar',
                                   name='foo')


rs = scn.resample(area_def)
rs.save_dataset(dst_id, filename='hv.tif')

# Bonus: TrollImage

PIL on steroids: PIL that can handle masked arrays + various colorspaces

[DOCS](https://trollimage.readthedocs.io)

In [ ]:
from trollimage.colormap import spectral, greys
from trollimage.image import Image

scn = Scene(platform_name="Terra", sensor="modis",
                base_dir=".",
                reader='hdfeos_l1b',
                start_time=datetime(2017, 8, 26, 14, 35),
                end_time=datetime(2017, 8, 26, 14, 45))

scn.load(['32'])
data = scn.datasets['32']

img = Image(data, mode="L")

greys.set_range(-20 + 273.15, -10 + 273.15)
spectral.set_range(-90 + 273.15, -20.00001 + 273.15)
my_cm = spectral + greys
img.colorize(my_cm)

img.show()

In [ ]:
from satpy.multiscene import MultiScene

In [ ]:
scn1 = Scene(platform_name="Terra", sensor="modis", 
                base_dir=".",
                reader='hdfeos_l1b',
                start_time=datetime(2017, 8, 26, 14, 35),
                end_time=datetime(2017, 8, 26, 14, 39))

scn2 = Scene(platform_name="Terra", sensor="modis",
                base_dir=".",
                reader='hdfeos_l1b',
                start_time=datetime(2017, 8, 26, 13, 0),
                end_time=datetime(2017, 8, 26, 13, 5))

scenes = [scn1, scn2]

mscn = MultiScene(scenes)
mscn.load(['true_color'])
mscn = mscn.resample('fram')
scn = mscn.blend()
scn.save_dataset('true_color', '/tmp/blend.png')
